In [1]:
import numpy as np
import matplotlib.pyplot as plt
import freud
import signac
import gsd
import hoomd
import gsd.pygsd
import gsd.hoomd
import scipy
from scipy import signal
import time
import os
from scipy.stats import linregress

In [2]:
data_path = "/home/gwenwhite/cmelab/ptb7-itic/ptb7-project/"
project = signac.get_project(data_path)

In [3]:
state_dict={"size": "small", 
            "process": "quench"}
            #"kT_reduced":1.1, 
            #"density":0.8}

job_list= project.find_jobs(state_dict)
#job_list= project
#specify file type
for job in job_list:
    msd_path= job.fn('msd.log')
    rdf_path= job.fn('trajectory.gsd')
    

In [4]:
def atom_type_pos(frame, atom_types): 
    positions = []
    for idx, type_id in enumerate(frame.particles.typeid):
        if frame.particles.types[type_id] in atom_types:
            positions.append(frame.particles.position[idx])
    return positions


def rdf_from_gsd(rdf_path, 
                 atom_types='all', 
                 r_max = None, 
                 r_min= 0.1, 
                 nbins= 50, 
                 start= 95):
    
    try: 
        f = gsd.pygsd.GSDFile(open(rdf_path, "rb"))
    except PermissionError:
        print("problem with", rdf_path)
        return None
    trajectory = gsd.hoomd.HOOMDTrajectory(f)
    if r_max is None:
        r_max = max(trajectory[-1].configuration.box[:3]) * 0.45
    freud_rdf = freud.density.RDF(bins=nbins, r_max=r_max, r_min=r_min)
    if len(trajectory[start:]) <1: 
        print('two fewer frames')
        return None
    for frame in trajectory[start:]:
        if atom_types == 'all':
            freud_rdf.compute(frame, reset=False)
        else:
            query_points = atom_type_pos(frame, atom_types)
            box = frame.configuration.box
            freud_rdf.compute(system=(box, query_points), reset=False)
            
    x = freud_rdf.bin_centers
    y = freud_rdf.rdf
    #Be aware of atom type naming with multiple atom types in function
    filename = 'gwen-{}-trajectory.txt'.format(atom_types[0])
    np.savetxt(filename, np.transpose([x,y]), delimiter=',', header= "bin_centers, rdf")
    f.close()
    print("it worked")
    return freud_rdf
        

def msd_slope(job_list, step_size = 2, atom_types='all'):  
    msd = np.genfromtxt(job_list, names=True, delimiter="\t")
    x = msd['timestep'][-3000::step_size]
    y = msd[atom_types][-3000::step_size]
    slope, intercept, r_value, p_value, std_err = linregress(x, y)
    plt.plot(x, y)
    plt.plot(x, x*slope + intercept, linestyle = 'dotted')
    plt.xlabel("timestep", fontsize=14)
    plt.ylabel("msd", fontsize=14)
    plt.show()         
    return x, y, slope, intercept
    

In [5]:
def msd_analysis(job_list):
    for job in job_list:
        if msd_path:
            x, y, slope, intercept = msd_slope(msd_path)

def rdf_analysis(job_list):
    for job in job_list:
        print(job)
        rdf_path= job.fn('trajectory.gsd')
        rdf_from_gsd(rdf_path, atom_types= ['f'], start =95)


In [6]:
rdf_analysis(job_list)


2244c441e021a5ca72d1d4cef64416f6
problem with /home/gwenwhite/cmelab/ptb7-itic/ptb7-project/workspace/2244c441e021a5ca72d1d4cef64416f6/trajectory.gsd
925cefa99ac87dc0cfd99f13e6e22f04
problem with /home/gwenwhite/cmelab/ptb7-itic/ptb7-project/workspace/925cefa99ac87dc0cfd99f13e6e22f04/trajectory.gsd
606e1f12f27149e5991f7f8278c6a5cd
problem with /home/gwenwhite/cmelab/ptb7-itic/ptb7-project/workspace/606e1f12f27149e5991f7f8278c6a5cd/trajectory.gsd
7052874ab3e0cff9890bd2243227a716
problem with /home/gwenwhite/cmelab/ptb7-itic/ptb7-project/workspace/7052874ab3e0cff9890bd2243227a716/trajectory.gsd
b86a895bc64bb4cf8a386e3326539f1a
it worked
e18546285aeef7bde68c1bda5fda3f72
problem with /home/gwenwhite/cmelab/ptb7-itic/ptb7-project/workspace/e18546285aeef7bde68c1bda5fda3f72/trajectory.gsd
a7284e3b01126eb99d6bee7245cd4802
problem with /home/gwenwhite/cmelab/ptb7-itic/ptb7-project/workspace/a7284e3b01126eb99d6bee7245cd4802/trajectory.gsd
f2e48e3760cf2c65a3a5e540d857bdfa
it worked
3fc1ab56b4806e

it worked
099207e44e80f9d76633de46287ad497
problem with /home/gwenwhite/cmelab/ptb7-itic/ptb7-project/workspace/099207e44e80f9d76633de46287ad497/trajectory.gsd
bb488d2b1ded83e3be4225f6952bdc28
it worked
e881c90d586d00aa59957db82e341628
it worked
a0086c37f23778c5620d4f56cec564d6
it worked
44d762ab52c28a6f4c62246781efc9f2
it worked
66827db87a0e491096a85d0b5adc7830
problem with /home/gwenwhite/cmelab/ptb7-itic/ptb7-project/workspace/66827db87a0e491096a85d0b5adc7830/trajectory.gsd
505ead7abc1c12458a1cfa91d51295e1
problem with /home/gwenwhite/cmelab/ptb7-itic/ptb7-project/workspace/505ead7abc1c12458a1cfa91d51295e1/trajectory.gsd
fc2c6afde4a39f237c2d8984b23c5aa9
it worked
69633e447ed6dd55193d2813d442d8b6
it worked
94bea6a1cac1435e2be860b4e6f725d8
problem with /home/gwenwhite/cmelab/ptb7-itic/ptb7-project/workspace/94bea6a1cac1435e2be860b4e6f725d8/trajectory.gsd
d9009caf9922d1d1403a3879ed362c38
it worked
b708b5ff868ed2dba7200146d77e90ba
it worked
ef2b780fd280b13fc455dd96b2a638c6
it worked
553

it worked
d6a89e9ce106ab76844cfe4038299129
problem with /home/gwenwhite/cmelab/ptb7-itic/ptb7-project/workspace/d6a89e9ce106ab76844cfe4038299129/trajectory.gsd
97d34e25e8df063aa700f1398b08867b
problem with /home/gwenwhite/cmelab/ptb7-itic/ptb7-project/workspace/97d34e25e8df063aa700f1398b08867b/trajectory.gsd
b1c87758280b63ccb026fceb3d404347
problem with /home/gwenwhite/cmelab/ptb7-itic/ptb7-project/workspace/b1c87758280b63ccb026fceb3d404347/trajectory.gsd
97d1d39623bd5468b6c0f9a5824f8085
it worked
03a7718cf38b3f884a22e356a3736e13
problem with /home/gwenwhite/cmelab/ptb7-itic/ptb7-project/workspace/03a7718cf38b3f884a22e356a3736e13/trajectory.gsd
eb43077c6346dc00d3285cdf64484727
it worked
2ddc3b7f30657250902937c85a3558c2
it worked
5e866be825f7ca0b63a5b0323edd78a2
it worked
24cbc6cd4c05df7712c9884d4c5e74b4
problem with /home/gwenwhite/cmelab/ptb7-itic/ptb7-project/workspace/24cbc6cd4c05df7712c9884d4c5e74b4/trajectory.gsd
b1d816218a78fde6fe7797406326c253
it worked
d470bd2bd6e3c3b7b1115843e

it worked
1a5487fe6f07e7c2a6692ed78c5cd123
problem with /home/gwenwhite/cmelab/ptb7-itic/ptb7-project/workspace/1a5487fe6f07e7c2a6692ed78c5cd123/trajectory.gsd
16d6f5e514bf64a8b364d81335d5e238
problem with /home/gwenwhite/cmelab/ptb7-itic/ptb7-project/workspace/16d6f5e514bf64a8b364d81335d5e238/trajectory.gsd
3fbd51c414cd0f025e27ae1323bcd653
it worked
f3a4382620b008ba3ffd22c37f591176
problem with /home/gwenwhite/cmelab/ptb7-itic/ptb7-project/workspace/f3a4382620b008ba3ffd22c37f591176/trajectory.gsd
8db543b006ce4ab430d4d8ba0b142b7d
it worked
368a15b5e272b4c736f9df8fdeb22868
it worked
450851cd1be23f1aa089d3e273660b39
it worked
39027ca0fa3a4b1f44d99a7de4cd2b8d
problem with /home/gwenwhite/cmelab/ptb7-itic/ptb7-project/workspace/39027ca0fa3a4b1f44d99a7de4cd2b8d/trajectory.gsd
b45af4ff0f9517823994bd70d23e9ade
it worked
b1f04daddbfd948328f534acdbe64fe2
it worked
6ea331d68cd0da92546c2b4c907b337e
problem with /home/gwenwhite/cmelab/ptb7-itic/ptb7-project/workspace/6ea331d68cd0da92546c2b4c907b33